# Data Collection

Explore the website thoroughly to gather relevant information on each data item for house price prediction.


1.   Website gồm nhiều trang, mỗi trang có 20 bài đăng bán nhà
2.   Nhóm duyệt qua từng trang, ở mỗi trang tiếp tục duyệt qua từng bài đăng bán nhà
3.   Nhóm lấy các trường dữ liệu từ trang web như sau
    * Title
    * Gmail
    * Area
    * Address
    * Seller Name
    * Number of bedrooms
    * Number of bathrooms
    * Price
    * Content
    * district


Determine the size of the dataset to be collected.
* Nhóm em cào 450 trang dữ liệu từ trang web nhưng chỉ lấy ở TP HCM. Vì dữ liệu trên web chủ yếu là thành phố HCM. Chúng em cào 450 trang là lấy được 9000 dòng dữ liệu. Các bạn dựa theo code ở dưới để chia ra cào

In [ ]:
import pandas as  pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request
import requests
import re
def download_html(url):
    with urllib.request.urlopen(url) as response:
        html = response.read()
        html = html.decode('utf-8')
    response.close()
    return html

In [ ]:
def scrape_data(link):
    url=link
    html = download_html(url)
    soup = BeautifulSoup(html, 'lxml')
    house={}
    #Lấy title
    try:
      title = soup.find('h1',class_='uk-panel-title').get_text()
      title = re.sub('\s+', ' ', title).strip()
      house['Title'] = title
    except:
      house['Title']=None
    try:
      # lấy tên người bán
      house['Seller Name']=soup.find('div',class_='name').find('a').get_text()
    except:
      house['Seller Name']=None
    try:
      # lấy  email người bán
      house['Seller Email']=soup.find('div',class_='more email').find('a').get('href').replace('mailto:','')
    except:
      house['Seller Email']=None
    try:
      # lấy giá nhà
      Gia = soup.find('strong',class_='price').get_text().strip()
      if(Gia == 'Thỏa thuận'):
        house['Area']=None
        house['Address']=None
        house['Title'] = None
        house['Seller Name']=None
        house['Seller Email']=None
        house['Number of bedrooms']=None
        house['Number of bathrooms']=None
        house['Price']=None
        house['Content']=None
        house['district']=None

        return house
      else:
        house['Price'] = Gia

    except:
      house['Price']=None

    try:
      # Lay ra quận, huyện
      house['district'] = soup.find('ul', class_='uk-breadcrumb').find_all('li')[-1].get_text()
    except:
      house['district'] = None

    try:
      # vào thẻ div class param
      param=soup.find_all('div',class_='param')[0]
    except:

      house['Area']=None
      house['Address']=None
      house['Number of bedrooms']=None
      house['Number of bathrooms']=None
      print('Here need checking')

    try :
      # lấy ra diện tích , address, số phòng ngủ và số phòng tắm từ tab param
      house['Area']=float(re.compile(r'<strong>Diện tích:</strong>\s*(.*?)\s*m<sup>2</sup>').search(str(param)).group(1))
    except:
      house['Area']=None
    try:
      house['Address']=re.compile(r'<strong>Địa chỉ:</strong>\s*(.*?)</li>').search(str(param)).group(1)
    except:
      house['Address']=None
    try:
      house['Number of bedrooms']=int(re.compile(r'<strong>Phòng ngủ:</strong>\s*(.*?)\s*PN').search(str(param)).group(1))
    except:
       house['Number of bedrooms']=None
    try:
      house['Number of bathrooms']=int(re.compile(r'<strong>Phòng WC:</strong>\s*(.*?)\s*WC').search(str(param)).group(1))
    except:
      house['Number of bathrooms']=None
      #vào thẻ div content
    try:
      content = soup.find('div', class_='content')

      # Loại bỏ tất cả các thẻ <br> và <p> trong phần tử đã tìm thấy
      for br in content.find_all("br"):
          br.replace_with("\n")
      for p in content.find_all("p"):
          p.unwrap()

      # Chuyển đổi nội dung đã xử lý thành chuỗi
      content_str = content.get_text()

      content_str = re.sub(r'[^\w\s,.!?]', '', content_str)

      # Loại bỏ các khoảng trắng thừa
      content_str = re.sub('\s+', ' ', content_str).strip()
      house['Content'] = content_str
    except:
      house['Content'] = None

    return house


In [ ]:
import time

def getItems(index,maxIndex,data):#22280069
  url='https://batdongsan.vn/ban-nha-ho-chi-minh/p1'
  url=url.replace('/p1','/p'+str(index))

  while index<maxIndex+1:
    try:
      print(f'-----------Crawling page {index}-----------')
      start_time = time.time()
      html=download_html(url)
      soup = BeautifulSoup(html, 'lxml')
      dataTab=soup.find('div',class_='datalist')
      itemLinks = [link.find('a').get('href') for link in dataTab.findAll('div',class_='name')]
      data_tempt=[scrape_data(link) for link in itemLinks]
      for i in data_tempt:
        data.append(i)
      # data.append(data_tempt)
    except Exception as e:
      print(f"An error occurred when crawling at page{index}:", e)
    url = url.replace('/p'+str(index),'/p'+str(index+1))
    end_time = time.time()
    print(f'Crawl page {index} successfully , time completed : {-start_time+end_time} s')
    index+=1

In [ ]:
data=[]
getItems(151,200,data)
print(len(data))

-----------Crawling page 151-----------
Crawl page 151 successfully , time completed : 72.79038214683533 s
-----------Crawling page 152-----------
Crawl page 152 successfully , time completed : 72.05754208564758 s
-----------Crawling page 153-----------
Crawl page 153 successfully , time completed : 69.74116444587708 s
-----------Crawling page 154-----------
Crawl page 154 successfully , time completed : 72.22955751419067 s
-----------Crawling page 155-----------
Crawl page 155 successfully , time completed : 74.0295820236206 s
-----------Crawling page 156-----------
Crawl page 156 successfully , time completed : 74.7363748550415 s
-----------Crawling page 157-----------
Crawl page 157 successfully , time completed : 73.77369117736816 s
-----------Crawling page 158-----------
Crawl page 158 successfully , time completed : 72.56320714950562 s
-----------Crawling page 159-----------
Crawl page 159 successfully , time completed : 72.34133696556091 s
-----------Crawling page 160-----------

In [ ]:
import json
with open('Data_151_200.json', "w",encoding="utf-8") as json_file:
    # Ghi thông tin vào tập tin JSON
    json.dump(data, json_file,ensure_ascii=False, indent=4)
df = pd.json_normalize(data)

# Save DataFrame to CSV
df.to_csv('Data_151_250.csv', index=False)

print("JSON data has been converted to CSV and saved as 'output.csv'")

JSON data has been converted to CSV and saved as 'output.csv'


Chúng em chia ra cho mỗi người chay trên từng máy tính khác nhau rồi sau đó gọp dữ liệu lại